In [1]:
import numpy as np
from TMDP import TMDP
from River_swim import River

from algorithms import *
from model_functions import *

import matplotlib.pyplot as plt
import gymnasium as gym
from FrozenLake import *

#np.set_printoptions(precision=4)
import math
from utils import *

nrows = 8
nS = nrows**2
nA = 4
seed = get_current_seed()
gamma = .9
tau = 1.
#tmdp = TMDP(river, xi, tau=tau, gamma=gamma, seed=seed)
#mdp = TMDP(river, xi, tau=0., gamma=gamma, seed=seed)

env = FrozenLakeEnv(is_slippery=False, seed=seed, desc=generate_random_map(nrows))#, render_mode="human")
xi_frozen = np.ones(env.nS) * 1/env.nS
env.reset()

Current seed for result reproducibility: 113797913031798016444128813178113627229


(0, {'prob': 1})

In [2]:
tmdp = TMDP(env, xi_frozen, tau=tau, gamma=gamma, seed=seed)

In [3]:
q_c = np.zeros((nS, nA))
cur_res = curriculum_MPI(tmdp, q_c, alpha = .25, episodes=1000000, batch_nS=1, temperature=20.)


[(0, 0.9943562610229277), (1, 0.9943562610229277)]
[-35.3875000000001, -35.3875000000001]
Updating the policy with alpha_star:  1 tau_star:  0.9943562610229277
[(0, 0.9887125220458555), (1, 0.9887125220458555)]
[-35.18750000000004, -35.18750000000004]
Updating the policy with alpha_star:  1 tau_star:  0.9887125220458555
[(0, 0.9830687830687832), (1, 0.9830687830687832)]
[-34.98750000000005, -34.98750000000005]
Updating the policy with alpha_star:  1 tau_star:  0.9830687830687832
[(0, 0.9774250440917109), (1, 0.9774250440917109)]
[-34.787499999999966, -34.787499999999966]
Updating the policy with alpha_star:  1 tau_star:  0.9774250440917109
[(0, 0.9717813051146387), (1, 0.9717813051146387)]
[-34.58749999999999, -34.58749999999999]
Updating the policy with alpha_star:  1 tau_star:  0.9717813051146387
[(0, 0.9661375661375664), (1, 0.9661375661375664)]
[-34.38750000000001, -34.38750000000001]
Updating the policy with alpha_star:  1 tau_star:  0.9661375661375664
[(0, 0.9604938271604941), (1

In [8]:
print(cur_res["Qs"][-1])
Q = cur_res["Qs"][-1]

[[-5.59966180e-02  2.72431361e-08  9.44426392e-02 -2.60487417e-02]
 [-1.26858620e-02  7.94895182e-19 -2.18283225e-04 -2.13812067e-02]
 [-1.62086801e-02  2.03565667e-03  1.56179268e-04 -1.47145913e-02]
 [ 2.40304310e-06 -7.50594567e-03  1.17967427e-01 -6.68779759e-03]
 [ 1.20234135e-06  3.47465848e-02  1.32398908e-01  7.68483150e-25]
 [ 1.07128614e-06  8.50473716e-07  2.61464211e-32  9.54515955e-07]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-3.01437274e-01  1.69978766e-06  8.03899398e-17  1.59767912e-17]
 [ 3.05759103e-08  3.43157309e-08  8.80367993e-03 -3.16508995e-02]
 [ 4.69582235e-10  1.57978404e-19  5.91502045e-10  9.88067309e-03]
 [ 0.00000000e+00  1.58870638e-18  3.37817799e-07 -1.08673588e-02]
 [-1.08355414e-02 -8.42418144e-03 -5.75042495e-02  2.49310288e-02]
 [ 8.74763175e-02 -6.45390005e-02  3.89972893e-02  3.09592070e-02]
 [ 7.79413989e-02  4.16048004e-11  2.18989323e-02  4.37680015e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e

In [5]:
status_step = 1000000
episodes = 1000000
batch_nS = 1
temperature = 20

In [6]:
done = False
Q_p = np.zeros((tmdp.nS, tmdp.nA))
Q_xi = np.zeros((tmdp.nS, tmdp.nA))
tau = 1.
while not done:
    if tau == 0:
        done = True
    batch_res = policy_gradient(tmdp, Q_p, Q_xi, alpha=.25, episodes=episodes, status_step=status_step, batch_nS=batch_nS, temperature=temperature)    
    Q_p = batch_res["Q_ps"][-1]
    Q_xi = batch_res["Q_xis"][-1]
    print("Updating tau from {} to {}".format(tau, max(0.,tau-0.05)))
    tau = max(0.,tau-0.05)
    tmdp.update_tau(tau)

In [ ]:
from utils import *
res = {"Q":batch_res["Qs"][-1], "Q_p":batch_res["Q_ps"][-1], "Q_xi":batch_res["Q_xis"][-1]}
json_res = to_json(res, indent=True)
write_results(nrows, nA, "FrozenLake", json_res, "w")


In [ ]:
json_res = read_results(nrows, nA, "FrozenLake")
res = from_json(json_res)
Q = np.array(res["Q"])

In [9]:

tmdp = TMDP(env, xi_frozen, tau=0., gamma=gamma, seed=seed)
env.render_mode = "human"
tmdp.reset()
done = False
step = 0
while True:
    a = greedy(tmdp.env.s, Q, tmdp.env.allowed_actions[int(tmdp.env.s)])
    s_prime, reward, flags, prob = tmdp.step(a)
    if flags["done"]:
        tmdp.reset()
    step +=1
    if step > max(100,nrows*2):
        break

In [ ]:
Q_0 = np.zeros((tmdp.nS, tmdp.nA))
env.render_mode = None
batch_res_0 = batch_q_learning(tmdp, Q_0, alpha=.25, episodes=episodes*21, status_step=status_step*21, batch_nS=batch_nS)    
Q_0 = batch_res_0["Qs"][-1]
    

In [ ]:
env.render_mode = "human"
tmdp.reset()
done = False
step = 0
while True:
    a = greedy(tmdp.env.s, Q_0, tmdp.env.allowed_actions[int(tmdp.env.s)])
    s_prime, reward, flags, prob = tmdp.step(a)
    if flags["done"]:
        tmdp.reset()
    step +=1
    if step > max(100,nrows*2):
        break